In [1]:
import pandas as pd
import numpy as np
import pickle
import scipy.optimize as optim
from sklearn import preprocessing
from sklearn.model_selection import train_test_split


Import the COMPAS dataset.

In [2]:
df = pd.read_csv('../data/compas-scores-two-years.csv')

In [3]:
df

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,8/14/2013,Male,4/18/1947,69,Greater than 45,Other,...,1,Low,8/14/2013,7/7/2014,7/14/2014,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,1/27/2013,Male,1/22/1982,34,25 - 45,African-American,...,1,Low,1/27/2013,1/26/2013,2/5/2013,0,9,159,1,1
2,4,ed philo,ed,philo,4/14/2013,Male,5/14/1991,24,Less than 25,African-American,...,3,Low,4/14/2013,6/16/2013,6/16/2013,4,0,63,0,1
3,5,marcu brown,marcu,brown,1/13/2013,Male,1/21/1993,23,Less than 25,African-American,...,6,Medium,1/13/2013,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,3/26/2013,Male,1/22/1973,43,25 - 45,Other,...,1,Low,3/26/2013,NaN,NaN,2,0,1102,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7209,10996,steven butler,steven,butler,11/23/2013,Male,7/17/1992,23,Less than 25,African-American,...,5,Medium,11/23/2013,11/22/2013,11/24/2013,0,1,860,0,0
7210,10997,malcolm simmons,malcolm,simmons,2/1/2014,Male,3/25/1993,23,Less than 25,African-American,...,5,Medium,2/1/2014,1/31/2014,2/2/2014,0,1,790,0,0
7211,10999,winston gregory,winston,gregory,1/14/2014,Male,10/1/1958,57,Greater than 45,Other,...,1,Low,1/14/2014,1/13/2014,1/14/2014,0,0,808,0,0
7212,11000,farrah jean,farrah,jean,3/9/2014,Female,11/17/1982,33,25 - 45,African-American,...,2,Low,3/9/2014,3/8/2014,3/9/2014,3,0,754,0,0


Data Preprocessing: From the ProPublica notebook, we removed the rows that 
1. charge date of a defendants Compas scored crime was not within 30 days from when the person was arrested
2. the recidivist flag - is_recid == -1 if we could not find a compas case at all
3. those with a c_charge_degree of 'O' which means ordinary traffic offenses. It will not result in Jail time are removed 
4. since we are only intersted in sample fairness between two races: African-American and Caucasian, we have subsets data

In [4]:
df = df[['age', 'c_charge_degree', 'race', 'age_cat', 'score_text', 'sex', 'priors_count', 'juv_fel_count','juv_misd_count','juv_other_count',
                    'days_b_screening_arrest', 'decile_score', 'is_recid', 'two_year_recid', 'c_jail_in', 'c_jail_out']]
ix = df['days_b_screening_arrest'] <= 30
ix = (df['days_b_screening_arrest'] >= -30) & ix
ix = (df['is_recid'] != -1) & ix
ix = (df['c_charge_degree'] != "O") & ix
ix = (df['score_text'] != 'N/A') & ix
df = df.loc[ix,:]
df['length_of_stay'] = (pd.to_datetime(df['c_jail_out'])-pd.to_datetime(df['c_jail_in'])).apply(lambda x: x.days)
df = df.loc[df['race'].isin(['African-American','Caucasian'])]
df = df[['race', 'sex', 'age', 'age_cat', 'c_charge_degree', 'score_text', 'priors_count', 'juv_fel_count','juv_misd_count','juv_other_count',
                    'days_b_screening_arrest', 'decile_score', 'length_of_stay', 'two_year_recid']]

#Since LFR needs to cageorize sensitive group and non-sensitive group,we regard defandant with African-American as 0,Caucasian as 1
df.loc[df['race'] == 'African-American', 'race']=0
df.loc[df['race'] == 'Caucasian', 'race']=1
df.loc[df['sex'] == 'Male', 'sex']=1
df.loc[df['sex'] == 'Female', 'sex']=0
df.loc[df['age_cat'] == 'Less than 25', 'age_cat']=0
df.loc[df['age_cat'] == '25 - 45', 'age_cat']=1
df.loc[df['age_cat'] == 'Greater than 45', 'age_cat']=2
df.loc[df['score_text'] == 'Low', 'score_text']=0
df.loc[df['score_text'] == 'Medium', 'score_text']=1
df.loc[df['score_text'] == 'High', 'score_text']=2
df.loc[df['c_charge_degree'] == 'F', 'c_charge_degree']=0
df.loc[df['c_charge_degree'] == 'M', 'c_charge_degree']=1
df

,race,sex,age,age_cat,c_charge_degree,score_text,priors_count,juv_fel_count,juv_misd_count,juv_other_count,days_b_screening_arrest,decile_score,length_of_stay,two_year_recid
1,0,1,34,1,0,0,0,0,0,0,-1.0,3,10,1
2,0,1,24,0,0,0,4,0,0,1,-1.0,4,1,1
6,1,1,41,1,0,1,14,0,0,0,-1.0,6,6,1
8,1,0,39,1,1,0,0,0,0,0,-1.0,1,2,0
10,1,1,27,1,0,0,0,0,0,0,-1.0,4,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7207,0,1,30,1,1,0,0,0,0,0,-1.0,2,0,1
7208,0,1,20,0,0,2,0,0,0,0,-1.0,9,0,0
7209,0,1,23,0,0,1,0,0,0,0,-1.0,7,1,0
7210,0,1,23,0,0,0,0,0,0,0,-1.0,3,1,0


In [5]:
data = np.array(df)
y = np.array(data[:,-1]).flatten()
data = data[:,:-1]
sensitive = data[:,0]




In [6]:
data = preprocessing.scale(data)
#data = data[:,:-0]
sensitive_idx = np.array(np.where(sensitive==1))[0].flatten()
nonsensitive_idx = np.array(np.where(sensitive!=1))[0].flatten()
data_sensitive = data[sensitive_idx,:]
data_nonsensitive = data[nonsensitive_idx,:]
y_sensitive = y[sensitive_idx]
y_nonsensitive = y[nonsensitive_idx]




In [7]:
training_sensitive,test_sensitive,ytrain_sensitive, ytest_sensitive = train_test_split(data_sensitive, y_sensitive, test_size = 0.15, random_state = 42)
training_nonsensitive,test_nonsensitive,ytrain_nonsensitive, ytest_nonsensitive = train_test_split(data_nonsensitive, y_nonsensitive, test_size = 0.15, random_state = 42)

training = np.concatenate((training_sensitive, training_nonsensitive))
ytrain = np.concatenate((ytrain_sensitive, ytrain_nonsensitive))

test = np.concatenate((test_sensitive, test_nonsensitive))
ytest = np.concatenate((ytest_sensitive, ytest_nonsensitive))
    


In [8]:
data_sensitive

array([[ 1.22871802,  0.4927064 ,  0.55837559, ...,  0.14213551,
         0.4847697 , -0.18996465],
       [ 1.22871802, -2.02960628,  0.38789451, ...,  0.14213551,
        -1.27517973, -0.27327099],
       [ 1.22871802,  0.4927064 , -0.63499194, ...,  0.14213551,
        -0.21921007, -0.29409758],
       ...,
       [ 1.22871802, -2.02960628, -0.37927033, ...,  0.33688921,
        -1.27517973, -0.29409758],
       [ 1.22871802,  0.4927064 , -0.97595409, ...,  0.14213551,
         1.18874948, -0.27327099],
       [ 1.22871802,  0.4927064 , -1.14643516, ...,  0.14213551,
         0.4847697 , -0.25244441]])

In [9]:
from numba import jit
import numpy as np
    
@jit
def distances(X, v, alpha, N, P, k):
    dists = np.zeros((N, P))
    for i in range(N):
        for p in range(P):
            for j in range(k):    
                dists[i, j] += (X[i, p] - v[j, p]) * (X[i, p] - v[j, p]) * alpha[p]
    return dists

@jit
def M_nk(dists, N, k):
    M_nk = np.zeros((N, k))
    exp = np.zeros((N, k))
    denom = np.zeros(N)
    for i in range(N):
        for j in range(k):
            exp[i, j] = np.exp(-1 * dists[i, j])
            denom[i] += exp[i, j]
        for j in range(k):
            if denom[i]:
                M_nk[i, j] = exp[i, j] / denom[i]
            else:
                M_nk[i, j] = exp[i, j] / 1e-6
    return M_nk

@jit    
def M_k(M_nk, N, k):
    M_k = np.zeros(k)
    for j in range(k):
        for i in range(N):
            M_k[j] += M_nk[i, j]
        M_k[j] /= N
    return M_k

@jit        
def x_n_hat(X, M_nk, v, N, P, k):
    x_n_hat = np.zeros((N, P))
    L_x = 0.0
    for i in range(N):
        for p in range(P):
            for j in range(k):
                x_n_hat[i, p] += M_nk[i, j] * v[j, p]
            L_x += (X[i, p] - x_n_hat[i, p]) * (X[i, p] - x_n_hat[i, p])
    return x_n_hat, L_x

@jit
def yhat(M_nk, y, w, N, k):
    yhat = np.zeros(N)
    L_y = 0.0
    for i in range(N):
        for j in range(k):
            yhat[i] += M_nk[i, j] * w[j]
        yhat[i] = 1e-6 if yhat[i] <= 0 else yhat[i]
        yhat[i] = 0.999 if yhat[i] >= 1 else yhat[i]
        L_y += -1 * y[i] * np.log(yhat[i]) - (1.0 - y[i]) * np.log(1.0 - yhat[i])
    return yhat, L_y

@jit
def LFR(params, data_sensitive, data_nonsensitive, y_sensitive, 
        y_nonsensitive,  k=10, A_x = 1e-4, A_y = 0.1, A_z = 1000, results=0):
    
    LFR.iters += 1 
    Ns, P = data_sensitive.shape
    Nns, _ = data_nonsensitive.shape
    
    alpha0 = params[:P]
    alpha1 = params[P : 2 * P]
    w = params[2 * P : (2 * P) + k]
    v = np.matrix(params[(2 * P) + k:]).reshape((k, P))
        
    dists_sensitive = distances(data_sensitive, v, alpha1, Ns, P, k)
    dists_nonsensitive = distances(data_nonsensitive, v, alpha0, Nns, P, k)

    M_nk_sensitive = M_nk(dists_sensitive, Ns, k)
    M_nk_nonsensitive = M_nk(dists_nonsensitive, Nns, k)
    
    M_k_sensitive = M_k(M_nk_sensitive, Ns, k)
    M_k_nonsensitive = M_k(M_nk_nonsensitive, Nns, k)
    
    L_z = 0.0
    for j in range(k):
        L_z += abs(M_k_sensitive[j] - M_k_nonsensitive[j])

    x_n_hat_sensitive, L_x1 = x_n_hat(data_sensitive, M_nk_sensitive, v, Ns, P, k)
    x_n_hat_nonsensitive, L_x2 = x_n_hat(data_nonsensitive, M_nk_nonsensitive, v, Nns, P, k)
    L_x = L_x1 + L_x2

    yhat_sensitive, L_y1 = yhat(M_nk_sensitive, y_sensitive, w, Ns, k)
    yhat_nonsensitive, L_y2 = yhat(M_nk_nonsensitive, y_nonsensitive, w, Nns, k)
    L_y = L_y1 + L_y2

    criterion = A_x * L_x + A_y * L_y + A_z * L_z

    if LFR.iters % 250 == 0:
        print(LFR.iters, criterion)
      
    if results:
        return yhat_sensitive, yhat_nonsensitive, M_nk_sensitive, M_nk_nonsensitive
    else:
        return criterion
LFR.iters = 0

In [10]:
k=5
rez = np.random.uniform(size=data.shape[1] * 2 + k + data.shape[1] * k)
yhat_sensitive, yhat_nonsensitive, M_nk_sensitive, M_nk_nonsensitive = LFR(rez,
                                                                           training_sensitive,
                                                                           training_nonsensitive,
                                                                           ytrain_sensitive,
                                                                           ytrain_nonsensitive,
                                                                           k, 1e-4, 0.1, 1000,
                                                                           results=1)
print(yhat_sensitive.shape)
print(yhat_nonsensitive.shape)
print(M_nk_sensitive.shape)
print(M_nk_nonsensitive.shape)

test_yhat_sensitive, test_yhat_nonsensitive, test_M_nk_sensitive, test_M_nk_nonsensitive = LFR(rez,
                                                                           test_sensitive,
                                                                           test_nonsensitive,
                                                                           ytest_sensitive,
                                                                           ytest_nonsensitive,
                                                                           k, 1e-4, 0.1, 1000,
                                                                           results = 1)


<ipython-input-9-451ec7726e1b>:61: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "LFR" failed type inference due to: non-precise type array(pyobject, 1d, C)
During: typing of argument at <ipython-input-9-451ec7726e1b> (65)

File "<ipython-input-9-451ec7726e1b>", line 65:
def LFR(params, data_sensitive, data_nonsensitive, y_sensitive, 
    <source elided>
    
    LFR.iters += 1 
    ^

  @jit
C:\ProgramData\Anaconda3\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "LFR" was compiled in object mode without forceobj=True.

File "<ipython-input-9-451ec7726e1b>", line 62:
@jit
def LFR(params, data_sensitive, data_nonsensitive, y_sensitive, 
^

  warnings.warn(errors.NumbaWarning(warn_msg,
C:\ProgramData\Anaconda3\lib\site-packages\numba\core\object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprec

(1787,)
(2698,)
(1787, 5)
(2698, 5)


In [11]:
bnd = []
for i, k2 in enumerate(rez):
    if i < data.shape[1] * 2 or i >= data.shape[1] * 2 + k:
        bnd.append((None, None))
    else:
        bnd.append((0, 1))

rez = optim.fmin_l_bfgs_b(LFR, x0=rez, epsilon=1e-5,
                          args=(training_sensitive, training_nonsensitive,
                                ytrain_sensitive, ytrain_nonsensitive, k, 1e-4,
                                0.1, 1000, 0),
                          bounds = bnd, approx_grad=True, maxfun=150000,
                          maxiter=150000, iprint=-1)


250 664.5356252165361
500 616.0661430483822
750 412.5172164276978
1000 410.1143607076729
1250 365.9392323406644
1500 327.99723375708385
1750 320.5872578944414
2000 381.9096144805358
2250 357.7764047036379
2500 310.8686783541526
2750 312.65951535584475
3000 311.73034590412857
3250 308.9046402919759
3500 309.83533391447446
3750 308.8675663484132
4000 308.7326865338468
4250 308.92380486516527
4500 308.73044205928505
4750 308.6781116738424
5000 308.80296990489705
5250 308.7242208543201
5500 308.6560651411925
5750 308.65232587652736
6000 308.6539977221692
6250 308.65211095243177
6500 308.6540588377994
6750 308.65269569375437
7000 308.65167301632295
7250 308.6501078897102
7500 308.6524525006643
7750 308.651571172064
8000 308.6540050649273
8250 308.6530066427186
8500 nan


In [12]:
w = rez[0][data.shape[1]*2:data.shape[1]*2+k]

v = rez[0][data.shape[1]*2+k:].reshape(k,data.shape[1])


ytest_sensitive_pred = test_M_nk_sensitive.dot(np.expand_dims(w,axis=1))
ytest_nonsensitive_pred = test_M_nk_nonsensitive.dot(np.expand_dims(w,axis=1))

ytest_sensitive = ytest_sensitive.flatten()
ytest_sensitive = list(ytest_sensitive)

ytest_nonsensitive = ytest_nonsensitive.flatten()
ytest_nonsensitive = list(ytest_nonsensitive)

target = ytest_sensitive + ytest_nonsensitive

ytest_sensitive_pred = ytest_sensitive_pred.flatten()
ytest_sensitive_pred = list(ytest_sensitive_pred)

ytest_nonsensitive_pred = ytest_nonsensitive_pred.flatten()
ytest_nonsensitive_pred = list(ytest_nonsensitive_pred)

pred = ytest_sensitive_pred + ytest_nonsensitive_pred

len(target),len(pred)



(793, 793)

In [13]:
pred

[0.5406518546504145,
 0.4947613530868877,
 0.584805051183606,
 0.49193164453952654,
 0.6283554640048371,
 0.6982407412653323,
 0.5716031574338065,
 0.5478971971404192,
 0.5848560338671589,
 0.6871565495350417,
 0.6178440802081949,
 0.48482073675119713,
 0.630818031518313,
 0.5432639084757926,
 0.5969443450973153,
 0.5410866074581661,
 0.5289111044036419,
 0.5437931225397439,
 0.5381321947046629,
 0.5507370318536767,
 0.5423853130989418,
 0.4799266081648419,
 0.6529842745744123,
 0.4243174595261098,
 0.5553534404094262,
 0.5933652531766389,
 0.4946039791745256,
 0.49311500492590565,
 0.5394458232106658,
 0.6052715049144047,
 0.4822572336971078,
 0.5470398889340105,
 0.5157557651704658,
 0.5386510927254821,
 0.5675337689743312,
 0.48316777970669256,
 0.5036664300458499,
 0.5369346558335933,
 0.5903758453636347,
 0.6293097520624706,
 0.44334686542460505,
 0.4897361661172492,
 0.5507127578597003,
 0.5288617767181413,
 0.4736621938651532,
 0.5397375841662415,
 0.5478971971404192,
 0.6110508

In [21]:
 M_nk_sensitive

array([[0.2, 0.2, 0.2, 0.2, 0.2],
       [0.2, 0.2, 0.2, 0.2, 0.2],
       [0.2, 0.2, 0.2, 0.2, 0.2],
       ...,
       [0.2, 0.2, 0.2, 0.2, 0.2],
       [0.2, 0.2, 0.2, 0.2, 0.2],
       [0.2, 0.2, 0.2, 0.2, 0.2]])